<a href="https://colab.research.google.com/github/AIbyMohini/Quantum-Health-Classifier/blob/main/Quantum_Classifier_Healthcare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Mohini_Gohel_Project - Quantum Health: A Quantum Machine Learning Classifier for Diabetes Prediction

"""In this project, I built a quantum classifier using the Pima Indians Diabetes dataset to predict
whether a patient has diabetes or not. We combined classical data preprocessing techniques like
PCA (Principal Component Analysis) with quantum circuits to explore the potential of
hybrid quantum-classical machine learning."""

# Install required libraries
# !pip install pennylane scikit-learn matplotlib pandas

import pennylane as qml
from pennylane import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pandas as pd


# Load the dataset (Pima Indians Diabetes dataset)
url = "https://raw.githubusercontent.com/plotly/datasets/master/diabetes.csv"
df = pd.read_csv(url)

# Use only 2 features for visualization simplicity: Glucose and BMI
X = df[['Glucose', 'BMI']].values
y = df['Outcome'].values  # 0 for non-diabetic, 1 for diabetic

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# Quantum device: 2 qubits
dev = qml.device("default.qubit", wires=2)

# Quantum circuit
@qml.qnode(dev)
def circuit(weights, x):
    qml.templates.AngleEmbedding(x, wires=[0, 1])
    qml.templates.StronglyEntanglingLayers(weights, wires=[0, 1])
    return qml.expval(qml.PauliZ(0))

# Loss function (square loss)
def cost(weights, bias, X, Y):
    predictions = np.array([circuit(weights, x) + bias for x in X])
    return np.mean((predictions - (2*Y - 1)) ** 2)

# Quantum classifier
def quantum_classifier(weights, bias, x):
    return np.sign(circuit(weights, x) + bias)

# Initialize parameters
num_layers = 3
weights = np.random.randn(num_layers, 2, 3, requires_grad=True)
bias = np.array(0.0, requires_grad=True)

# Optimizer
opt = qml.GradientDescentOptimizer(stepsize=0.2)

# Training loop
epochs = 30
for epoch in range(epochs):
    # Update parameters
    weights, bias = opt.step(lambda w, b: cost(w, b, X_train, y_train), weights, bias)

    # Compute current cost
    c = cost(weights, bias, X_train, y_train)

    if (epoch+1) % 5 == 0:
        print(f"Epoch {epoch+1}: cost={c:.4f}")

# Predictions
y_pred = [quantum_classifier(weights, bias, x) for x in X_test]
y_pred = (np.array(y_pred) > 0).astype(int)

# Accuracy
acc = accuracy_score(y_test, y_pred)
print(f"Test accuracy: {acc:.2f}")


Epoch 5: cost=0.7711
Epoch 10: cost=0.7251
Epoch 15: cost=0.7037
Epoch 20: cost=0.6932
Epoch 25: cost=0.6872
Epoch 30: cost=0.6833
Test accuracy: 0.73


In [5]:
from sklearn.decomposition import PCA

# Apply PCA to reduce to 2 principal components
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

# Standardize the PCA features
X_pca_scaled = scaler.fit_transform(X_pca)

# Train-test split again
X_train, X_test, y_train, y_test = train_test_split(
    X_pca_scaled, y, test_size=0.2, random_state=42
)

print("Shape after PCA:", X_pca_scaled.shape)


Shape after PCA: (768, 2)


In [ ]:
import matplotlib.pyplot as plt

# Create a meshgrid to plot decision boundaries
h = 0.02
x_min, x_max = X_pca_scaled[:, 0].min() - 1, X_pca_scaled[:, 0].max() + 1
y_min, y_max = X_pca_scaled[:, 1].min() - 1, X_pca_scaled[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

# Flatten grid
grid = np.c_[xx.ravel(), yy.ravel()]

# Predict on the grid
Z = [quantum_classifier(weights, bias, point) for point in grid]
Z = (np.array(Z) > 0).astype(int)
Z = Z.reshape(xx.shape)

# Plot decision boundary
plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.8)
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, edgecolors='k', cmap=plt.cm.coolwarm)
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.title('Quantum Classifier Decision Boundary')
plt.show()
